In [13]:
#import secrets
import requests
import json

#def get_new_code_verifier() -> str:
#    token = secrets.token_urlsafe(100)
#    return token[:128]

#code_verifier = code_challenge = get_new_code_verifier()

client_id = "d1ba6c8f42d9abdc2aa2d84fb61702e5"
#base_url = "https://myanimelist.net/v1/oauth2/authorize"
#response_type = "code"
#redirect_url = "" #unimplemented

base_url = "https://api.myanimelist.net/v2/"
auth_header = {"X-MAL-CLIENT-ID" : client_id}

In [18]:
# Get Anime List of turtlemage

user = "turtlemage"
content_type = "animelist"

url = base_url + "users/" + user + "/" + content_type

fields = {
    "fields" : "list_status"
}

r = requests.get(url, headers = auth_header, params = fields)
print(r.status_code)

full_json = r.json()
#next_request_url = full_json['paging']['next']
print(full_json)
while 'paging' in full_json:
    paging = full_json['paging']
    new_url = paging['next']
    r = requests.get(new_url, headers = auth_header)
    if r.status_code == 200:
        full_json['data'] += r.json()['data']
        if 'next' in r.json()['paging']:
            full_json['paging'] = r.json()['paging']
        else:
            full_json.pop('paging')
print(full_json)


200
{'data': [{'node': {'id': 38101, 'title': '5-toubun no Hanayome', 'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/1819/97947.jpg', 'large': 'https://api-cdn.myanimelist.net/images/anime/1819/97947l.jpg'}}, 'list_status': {'status': 'dropped', 'score': 4, 'num_episodes_watched': 6, 'is_rewatching': False, 'updated_at': '2021-01-27T02:13:39+00:00'}}, {'node': {'id': 41457, 'title': '86', 'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/1987/117507.jpg', 'large': 'https://api-cdn.myanimelist.net/images/anime/1987/117507l.jpg'}}, 'list_status': {'status': 'plan_to_watch', 'score': 0, 'num_episodes_watched': 0, 'is_rewatching': False, 'updated_at': '2022-01-20T06:52:31+00:00'}}, {'node': {'id': 32998, 'title': '91 Days', 'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/13/80515.jpg', 'large': 'https://api-cdn.myanimelist.net/images/anime/13/80515l.jpg'}}, 'list_status': {'status': 'completed', 'score': 9, 'num_epis

In [68]:
import requests
import json
import urllib
def get_user_list_ids(client_id, user, content_type='anime', fields='list_status', returnJSON=True, return_type='id'):
    """
    Parameters:
    
    client_id    - Client ID generated by MyAnimeList app ("www.myanimelist.net/apiconfig/")
    user         - username of user
    content_type - anime or manga, to decide whether to find animelist or mangalist respectively
    fields       - list of fields to include in returned data, string deliminated by commas
    returnJSON   - boolean deciding whether to return raw JSON or to return simplified data
    return_type  - recommended id or title, determines the keys of formatted dictionary        
    """ 
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    #contentType is either anime or manga
    if content_type == 'anime':
        content = 'animelist'
    else:
        content = 'mangalist'
    data_request = {'fields' : fields}
    # Create request url, to pass HTTP GET request with header and fields
    url = base_url + "users/" + user + "/" + content
    # Initial HTTP request
    r = requests.get(url, headers = auth_header, params = data_request)
    if r.status_code != 200:
        raise Exception("HTTP GET Error : "+str(r.status_code))
    full_json = r.json()
    while 'paging' in full_json:
        paging = full_json['paging']
        new_url = paging['next']
        r = requests.get(new_url, headers = auth_header)
        if r.status_code != 200:
            raise Exception("HTTP GET Error : "+str(r.status_code))
        full_json['data'] += r.json()['data']
        if 'next' in r.json()['paging']:
            full_json['paging'] = r.json()['paging']
        else:
            full_json.pop('paging')
    if returnJSON:
        return full_json
    else:
        shows = full_json['data']
    field_list = fields.split(',')
    show_list = {}
    for node in full_json['data']:
        try:
            show = node['node'][return_type]
            show_list[show] = {}
        except:
            raise Exception('return_type not valid - must be id, title, or other field type')
        for field in field_list:
            show_list[show][field] = node['node'][field]
    return show_list
def get_show_id(client_id, show_title, content_type='anime'):
    if type(show_title) is list:
        show_ids = []
        for show in show_title:
            show_id.append(get_show_id(client_id, show, content_type))
        return show_ids
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    #contentType is either anime or manga
    if content_type == 'anime':
        content = 'anime'
    else:
        content = 'manga'
    # Create request url, to pass HTTP GET request with header and fields
    url = base_url + content + "?q=" +  urllib.parse.quote(show_title.lower().split()[0])
    # Initial HTTP request
    r = requests.get(url,headers = auth_header)
    if r.status_code != 200:
        raise Exception("HTTP GET Error : "+str(r.status_code))
    for node in r.json()['data']:
        if node['node']['title'] == show_title:
            return node['node']['id']
    raise Exception("Title not Found")
def get_show_stats(client_id, show_id=-1, show_title=-1, content_type='anime', stat='all'):
    if show_id == -1 and show_title == -1:
        raise Exception("Must submit an id or title")
    elif show_id == -1:
        show_title = get_show_id(client_id, show_title, content_type)
    auth_header = {'X-MAL-CLIENT-ID' : client_id}
    base_url = "https://api.myanimelist.net/v2/"
    #contentType is either anime or manga
    if content_type == 'anime':
        content = 'anime/'
    else:
        content = 'manga/'
    # Create request url, to pass HTTP GET request with header and fields
    url = base_url + content + show_id
    # Initial HTTP request
    r = requests.get(url, headers = auth_header, params = data_request)
    if stat == 'all':
        return r.json()
    else:
        return r.json()[stat]

In [69]:
animelist = get_user_list_ids(client_id, 'turtlemage')
print(animelist)
print(get_show_id(client_id, "Yuukoku no Moriarty"))

{'data': [{'node': {'id': 38101, 'title': '5-toubun no Hanayome', 'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/1819/97947.jpg', 'large': 'https://api-cdn.myanimelist.net/images/anime/1819/97947l.jpg'}}, 'list_status': {'status': 'dropped', 'score': 4, 'num_episodes_watched': 6, 'is_rewatching': False, 'updated_at': '2021-01-27T02:13:39+00:00'}}, {'node': {'id': 41457, 'title': '86', 'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/1987/117507.jpg', 'large': 'https://api-cdn.myanimelist.net/images/anime/1987/117507l.jpg'}}, 'list_status': {'status': 'plan_to_watch', 'score': 0, 'num_episodes_watched': 0, 'is_rewatching': False, 'updated_at': '2022-01-20T06:52:31+00:00'}}, {'node': {'id': 32998, 'title': '91 Days', 'main_picture': {'medium': 'https://api-cdn.myanimelist.net/images/anime/13/80515.jpg', 'large': 'https://api-cdn.myanimelist.net/images/anime/13/80515l.jpg'}}, 'list_status': {'status': 'completed', 'score': 9, 'num_episodes

40911
